<a href="https://colab.research.google.com/github/ish-war/DL-pima-diabetes-ann/blob/main/pima_indians_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('/content/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# relation bet variables and outcome col
# we can remove bloodpressure and skin thickness columns for better model
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [7]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
x = scaler.fit_transform(x)

In [10]:
x.shape

(768, 8)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2, random_state= 42)

In [86]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout


In [15]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim= 8))
model.add(Dense(1, activation= 'sigmoid'))

model.compile(loss= 'binary_crossentropy', optimizer= 'Adam', metrics= ['accuracy'])


In [18]:
model.fit(x_train, y_train, epochs= 100, batch_size= 32, validation_data= (x_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7934 - loss: 0.4275 - val_accuracy: 0.7532 - val_loss: 0.5042
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7821 - loss: 0.4362 - val_accuracy: 0.7532 - val_loss: 0.5050
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7646 - loss: 0.4740 - val_accuracy: 0.7403 - val_loss: 0.5053
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7782 - loss: 0.4848 - val_accuracy: 0.7403 - val_loss: 0.5061
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8092 - loss: 0.4194 - val_accuracy: 0.7468 - val_loss: 0.5060
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7938 - loss: 0.4388 - val_accuracy: 0.7468 - val_loss: 0.5056
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7994 - loss: 0.4478 - val_accuracy: 0.7468 - val_loss: 0.5065
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7953 - loss: 0.4498 - val_accuracy: 0.7532 - 

In [ ]:
# now we will learn how to do automate to find the best parameters that we added in model.add
 # using keras tuner

# 1. how to select appropriate optimizer
# 2. no of nodes in a layer
# 3. how to select no of hidden layers
# 4. all in all one model

In [19]:
!pip  install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.9 MB/s eta 0:00:00


In [21]:
# 1. how to select appropriate optimizer

# here we create tuner object and then pass functions inside that tuner object
import keras_tuner as kt


In [40]:
def build_model (hp) :
  model = Sequential()

  model.add(Dense(32, activation= 'relu', input_dim = 8))
  model.add(Dense(1, activation= 'sigmoid'))

  optimizer = hp.Choice('optimizer', values= ['adam','sgd','rmsprop', 'adadelta'])

  model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics= ['accuracy'] )

  return model

In [41]:
# now we are creating tuner object

tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials= 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [42]:
tuner.search(x_train, y_train, epochs= 5, validation_data=(x_test, y_test))

In [46]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [47]:
model = tuner.get_best_models(num_models= 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [48]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
model.fit(x_train, y_train, batch_size= 32, epochs= 100, initial_epoch= 6, validation_data= (x_test, y_test))


Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7431 - loss: 0.4943 - val_accuracy: 0.7143 - val_loss: 0.5362
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7525 - loss: 0.4922 - val_accuracy: 0.7208 - val_loss: 0.5297
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7491 - loss: 0.4999 - val_accuracy: 0.7208 - val_loss: 0.5254
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7816 - loss: 0.4788 - val_accuracy: 0.7208 - val_loss: 0.5207
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7711 - loss: 0.4701 - val_accuracy: 0.7208 - val_loss: 0.5170
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7847 - loss: 0.4562 - val_accuracy: 0.7208 - val_loss: 0.5168
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7749 - loss: 0.4645 - val_accuracy: 0.7273 - val_loss: 0.5154
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7674 - loss: 0.4563 - val_accuracy: 0.73

In [61]:
# 2. no of nodes in a layer

def build_model (hp) :
  model = Sequential()

  units = hp.Int('units', min_value= 8, max_value= 128, step= 8)

  model.add(Dense(units= units, activation= 'relu', input_dim = 8))
  model.add(Dense(1, activation= 'sigmoid'))

  model.compile(optimizer= 'rmsprop', loss = 'binary_crossentropy', metrics= ['accuracy'] )

  return model

In [62]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials= 5,
                        directory= 'mydir',
                        project_name= 'keras project')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
tuner.search(x_train, y_train, epochs= 5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 14s


In [64]:
tuner.get_best_hyperparameters()[0].values

{'units': 24}

In [65]:
model = tuner.get_best_models(num_models= 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [66]:
model.fit(x_train, y_train, batch_size= 32, epochs= 100, initial_epoch= 6, validation_data= (x_test, y_test))


Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7559 - loss: 0.5428 - val_accuracy: 0.7662 - val_loss: 0.5396
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7637 - loss: 0.5193 - val_accuracy: 0.7597 - val_loss: 0.5308
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7632 - loss: 0.4972 - val_accuracy: 0.7597 - val_loss: 0.5237
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7823 - loss: 0.4959 - val_accuracy: 0.7532 - val_loss: 0.5179
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7863 - loss: 0.4742 - val_accuracy: 0.7662 - val_loss: 0.5129
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7925 - loss: 0.4575 - val_accuracy: 0.7662 - val_loss: 0.5109
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7539 - loss: 0.4881 - val_accuracy: 0.7727 - val_loss: 0.5100
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7808 - loss: 0.4638 - val_accuracy: 0.77

In [67]:
# 3. select no of layers

def build_model (hp) :
  model = Sequential()

  model.add(Dense(72, activation= 'relu', input_dim = 8))

  for i in range(hp.Int('num_layers', min_value= 1, max_value= 100)) :
    model.add(Dense(72, activation= 'relu'))

  model.add(Dense(1, activation= 'sigmoid'))

  model.compile(optimizer= 'rmsprop', loss = 'binary_crossentropy', metrics= ['accuracy'] )

  return model


In [68]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials= 5,
                        directory= 'mydir',
                        project_name= 'num_layers')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [69]:
tuner.search(x_train, y_train, epochs= 5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 01m 11s


In [70]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 17}

In [71]:
model = tuner.get_best_models(num_models= 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 40 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [72]:
model.fit(x_train, y_train, batch_size= 32, epochs= 100, initial_epoch= 6, validation_data= (x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.7882 - loss: 0.4822 - val_accuracy: 0.7857 - val_loss: 0.5830
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8085 - loss: 0.4451 - val_accuracy: 0.7403 - val_loss: 0.5715
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7866 - loss: 0.4465 - val_accuracy: 0.7338 - val_loss: 0.6853
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7776 - loss: 0.4983 - val_accuracy: 0.7143 - val_loss: 0.6193
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8266 - loss: 0.4100 - val_accuracy: 0.6818 - val_loss: 0.7227
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7786 - loss: 0.4383 - val_accuracy: 0.7208 - val_loss: 0.6355
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8353 - loss: 0.4224 - val_accuracy: 0.7338 - val_loss: 0.7771
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8506 - loss: 0.4057 - val_accuracy: 0.72

In [87]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value= 1, max_value= 10 )) :
    if counter == 0 :
      model.add(Dense(hp.Int('units' + str(i), min_value= 8, max_value= 128, step= 8),
                      activation= hp.Choice('activation' + str(i),
                                            values= ['relu', 'tanh', 'sigmoid']),
                                            input_dim= 8
                      )
               )
      model.add(Dropout(hp.Choice('dropout' + str(i), values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else :
      model.add(Dense(hp.Int('units' + str(i), min_value= 8, max_value= 128, step= 8),
                      activation= hp.Choice('activation' + str(i),
                                            values= ['relu', 'tanh', 'sigmoid']
                                            )
                      )
               )
      model.add(Dropout(hp.Choice('dropout' + str(i), values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter += 1

  model.add(Dense(1, activation= 'sigmoid'))

  model.compile(optimizer= hp.Choice('optimizer', values= ['adam', 'sgd', 'rmsprop', 'adadelta']),
                loss= 'binary_crossentropy',
                metrics= ['accuracy']
                )

  return model

In [89]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials= 3,
                        directory= 'mydir',
                        project_name= 'final1')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [90]:
tuner.search(x_train, y_train, epochs= 5, validation_data= (x_test, y_test))


Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 13s


In [91]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 96,
 'activation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1}

In [92]:
model = tuner.get_best_models(num_models= 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [93]:
model.fit(x_train, y_train, epochs= 100, initial_epoch= 5, validation_data= (x_test, y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7587 - loss: 0.4937 - val_accuracy: 0.7662 - val_loss: 0.5142
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7807 - loss: 0.4640 - val_accuracy: 0.7597 - val_loss: 0.5102
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7859 - loss: 0.4623 - val_accuracy: 0.7597 - val_loss: 0.5117
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7849 - loss: 0.4750 - val_accuracy: 0.7532 - val_loss: 0.5109
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7986 - loss: 0.4443 - val_accuracy: 0.7532 - val_loss: 0.5125
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7990 - loss: 0.4469 - val_accuracy: 0.7532 - val_loss: 0.5118
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7859 - loss: 0.4741 - val_accuracy: 0.7532 - val_loss: 0.5113
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7470 - loss: 0.4802 - val_accuracy: 0.746

In [94]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 96)                  │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 96)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,949 (19.34 KB)

 Trainable params: 1,649 (6.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,300 (12.89 KB)